In [1]:
from IPython.display import display
import arcgis
from arcgis.gis import GIS
import pandas as pd

In [2]:
gis = GIS("https://datascienceqa.esri.com/portal", "portaladmin", "esri.agp", verify_cert=False)

In [3]:
search_result = gis.content.search('title:medicare_clean_data_geocoded_locations', 'Feature Layer')

In [4]:
medicare_data_item = search_result[0]
medicare_data_item

<Item title:"medicare_clean_data_geocoded_locations" type:Feature Layer Collection owner:portaladmin>

In [5]:
# Check layers
medicare_data_item.layers

[<FeatureLayer url:"https://datascienceqa.esri.com/server/rest/services/Hosted/medicare_clean_data_geocoded_locations/FeatureServer/0">]

In [6]:
# Get the layer
medicare_data_layer = medicare_data_item.layers[0]
medicare_data_layer

<FeatureLayer url:"https://datascienceqa.esri.com/server/rest/services/Hosted/medicare_clean_data_geocoded_locations/FeatureServer/0">

In [7]:
# Check properties of fields
medicare_data_layer.properties.fields

[{
   "name": "user_npi",
   "alias": "user_npi",
   "type": "esriFieldTypeDouble",
   "domain": null,
   "nullable": true,
   "editable": true
 }, {
   "name": "user_gender",
   "alias": "user_gender",
   "type": "esriFieldTypeString",
   "domain": null,
   "length": 1000000,
   "nullable": true,
   "editable": true
 }, {
   "name": "user_entity_type",
   "alias": "user_entity_type",
   "type": "esriFieldTypeString",
   "domain": null,
   "length": 1000000,
   "nullable": true,
   "editable": true
 }, {
   "name": "user_addressline1",
   "alias": "user_addressline1",
   "type": "esriFieldTypeString",
   "domain": null,
   "length": 1000000,
   "nullable": true,
   "editable": true
 }, {
   "name": "user_addressline2",
   "alias": "user_addressline2",
   "type": "esriFieldTypeString",
   "domain": null,
   "length": 1000000,
   "nullable": true,
   "editable": true
 }, {
   "name": "user_city",
   "alias": "user_city",
   "type": "esriFieldTypeString",
   "domain": null,
   "length": 1

In [8]:
# Print field names and type
for field in medicare_data_layer.properties.fields:
    print(field['name'],'          ',field['type'])

user_npi            esriFieldTypeDouble
user_gender            esriFieldTypeString
user_entity_type            esriFieldTypeString
user_addressline1            esriFieldTypeString
user_addressline2            esriFieldTypeString
user_city            esriFieldTypeString
user_postal_code            esriFieldTypeDouble
user_state            esriFieldTypeString
user_country            esriFieldTypeString
user_provider_type            esriFieldTypeString
user_medicare_participation_indicator            esriFieldTypeString
user_place_of_service            esriFieldTypeString
user_hcpcs_code            esriFieldTypeDouble
user_hcpcs_description            esriFieldTypeString
user_hcpcs_drug_indicator            esriFieldTypeString
user_bene_unique_cnt            esriFieldTypeDouble
user_bene_day_srvc_cnt            esriFieldTypeDouble
user_avg_medicare_allowed_amt            esriFieldTypeDouble
user_avg_submitted_chrg_amt            esriFieldTypeDouble
user_avg_medicare_payment_amt           

#### Create dataframe for WY

In [7]:
%time wy_df = medicare_data_layer.query(where="user_state='WY'", as_df=True)
wy_df.shape

Wall time: 37.8 s


(78343, 86)

In [13]:
# Check records that are outside WY
len(wy_df[wy_df['Region']!='Wyoming'])

16423

#### Create dataframe for ID

In [12]:
%time id_df = medicare_data_layer.query(where="user_state='ID'", as_df=True)
id_df.shape

Wall time: 1min 57s


(192853, 86)

In [14]:
# Check records that are outside WY
len(id_df[id_df['Region']!='Idaho'])

21801

In [9]:
map1 = gis.map('USA')
map1

MapView(layout=Layout(height='400px', width='100%'))

In [10]:
wy_df.spatial.plot(map1)

True

In [37]:
map2 = gis.map('Wyoming')
map2

MapView(layout=Layout(height='400px', width='100%'))

In [39]:
map2.remove_layers()

True

In [40]:
map2.add_layer({"type":"FeatureLayer", 
               "url":"https://datascienceqa.esri.com/server/rest/services/Hosted/medicare_clean_data_geocoded_locations/FeatureServer/0", 
               "definition_expression" : "Region='Wyoming'",
               "renderer":"ClassedSizeRenderer",
               "field_name":"user_outofpocket_insurance",
               "opacity":0.7
              })